In [ ]:
!pip install -q flwr[simulation] torch torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.


In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import warnings

warnings.filterwarnings('ignore')

import zipfile

from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST


import flwr as fl
from flwr.common import Metrics

import numpy as np
import pandas as pd
import random
import cv2
import os
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score



In [ ]:

CLASSES = (
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
)

NUM_CLIENTS = 10
BATCH_SIZE = 32

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import numpy as np
from sklearn.decomposition import PCA

def determine_pca_components(public_dataset, n_components):
    # Reshape the public dataset to 2D (num_samples, num_features)
    public_data = public_dataset.reshape(public_dataset.shape[0], -1)

    # Fit PCA on the public dataset
    pca = PCA(n_components=n_components)
    pca.fit(public_data)

    return pca

In [ ]:

def apply_pca_locally(data, pca, n_components):
    assert data is not None, "Data cannot be None"
    assert n_components is not None and isinstance(n_components, int), "n_components must be a valid integer"

    # Reshape the data to 2D (num_samples, num_features)
    data_2d = data.reshape(data.shape[0], -1)

    # Apply PCA transformation
    data_transformed = pca.transform(data_2d)[:, :n_components]

    # Calculate the percentage of data reduced or removed
    original_size = data_2d.size
    transformed_size = data_transformed.size
    data_removed_percentage = (1 - transformed_size / original_size) * 100

    # Reshape the transformed data to (num_samples, 1, n_components)
    data_transformed = data_transformed.reshape(-1, 1, n_components)

    return data_transformed, data_removed_percentage

In [ ]:
from torchvision.datasets import MNIST

# Define the transform to preprocess the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

mnist_dataset = MNIST("./dataset", train=True, download=True, transform=transform)

# Load the MNIST dataset
trainset = MNIST(root='./data', train=True, download=True, transform=transform)
testset = MNIST(root='./data', train=False, download=True, transform=transform)


100%|██████████| 9912422/9912422 [00:00<00:00, 102256645.02it/s]


Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz to ./dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 13413320.10it/s]

Extracting ./dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 29814246.04it/s]


...

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 32247480.44it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3850147.29it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, Subset
import random

def custom_distributed_split(indices, num_clients, class_idx, distribution_ratio):
    data_len = len(indices)
    class_data_range = int(data_len * distribution_ratio)
    non_class_data_range = int((data_len * (1 - distribution_ratio)) // (num_clients-1))

    out = []
    non_class_data_start = class_data_range + 1

    for i in range(num_clients):
        if i == class_idx:
            out.append(indices[:class_data_range + 1])
        else:
            non_class_data_end = non_class_data_start + non_class_data_range + 1
            out.append(indices[non_class_data_start : non_class_data_end])
            non_class_data_start += non_class_data_range

    if non_class_data_end != data_len:
        clients = [i for i in range(num_clients)]
        clients.pop(class_idx)
        for i in range(non_class_data_end, data_len):
            rand_class_idx = random.choice(clients)
            out[rand_class_idx].append(i)

    return out

def distribute_data_non_iid(dataset, num_clients):
    class_indices = [[] for _ in range(10)]  # 10 classes in MNIST
    for idx, (_, label) in enumerate(dataset):
        class_indices[label].append(idx)

    client_data_indices = [[] for _ in range(num_clients)]
    for class_idx, indices in enumerate(class_indices):
        np.random.shuffle(indices)
        split = custom_distributed_split(indices, num_clients, class_idx, 0.6)
        for client_idx in range(num_clients):
            client_data_indices[client_idx].extend(split[client_idx])

    client_datasets = [Subset(dataset, indices) for indices in client_data_indices]
    return client_datasets


In [ ]:
def determine_compression_parameters(client_data_variabilities):
    avg_variability = np.mean(client_data_variabilities)
    pruning_amount = np.interp(avg_variability, [0, 1], [0.2, 0.8])
    quantization_type = 'dynamic' if avg_variability > 0.5 else 'static'
    return pruning_amount, quantization_type

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, random_split, Subset

def determine_pca_components(public_dataset, client_data_variabilities, n_components_range=(8, 128)):
    # Reshape the public dataset to 2D (num_samples, num_features)
    public_data = public_dataset.reshape(public_dataset.shape[0], -1)

    # Determine the number of PCA components based on client data variability
    avg_variability = np.mean(client_data_variabilities)
    n_components = int(np.interp(avg_variability, [0, 1], n_components_range))

    # Fit PCA on the public dataset
    pca = PCA(n_components=n_components)
    pca.fit(public_data)

    return pca, n_components, avg_variability

def characterize_client_data(data):
    # Reshape the data to 2D (num_samples, num_features)
    data_2d = data.reshape(data.shape[0], -1)

    # Calculate feature variances
    feature_variances = np.var(data_2d, axis=0)

    # Calculate class distribution
    unique_classes, class_counts = np.unique(data_2d, return_counts=True)
    class_distribution = class_counts / np.sum(class_counts)

    # Calculate overall data variability
    data_variability = np.mean(feature_variances) * np.sum(class_distribution ** 2)

    return data_variability


In [ ]:
pca_components_range = [8, 16, 32, 64, 128]
#pca_components_range = [8, 16, 32, 64]

In [ ]:

DEVICE = torch.device("cpu")


def load_datasets(pca, n_components):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    trainset = MNIST("./dataset", train=True, download=True, transform=transform)
    testset = MNIST("./dataset", train=False, download=True, transform=transform)

    trainset_transformed, train_removed_percentage = apply_pca_locally(trainset.data.numpy(), pca, n_components)
    testset_transformed, test_removed_percentage = apply_pca_locally(testset.data.numpy(), pca, n_components)

    # Determine compression parameters based on client data variability
    avg_variability = np.mean(client_data_variabilities)
    pruning_amount = np.interp(avg_variability, [0, 1], [0.2, 0.8])
    quantization_type = 'dynamic' if avg_variability > 0.5 else 'static'

    # Split the dataset into non-IID client datasets
    client_datasets = distribute_data_non_iid(trainset, NUM_CLIENTS)

    trainloaders = []
    valloaders = []
    for ds in client_datasets:
        len_val = len(ds) // 10
        len_train = len(ds) - len_val
        ds_train, ds_val = random_split(ds, [len_train, len_val], generator=torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))

    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader, client_datasets, testset, train_removed_percentage, test_removed_percentage
    #return trainloaders, valloaders, testloader, client_datasets, testset, train_removed_percentage, test_removed_percentage, pruning_amount, quantization_type

def train_and_evaluate(pca, n_components, trainloader, valloader, testloader):
    net = Net().to(DEVICE)
    net, early_stopping_epoch, test_loss, test_accuracy = train(net, trainloader, valloader, testloader, epochs=100, device=DEVICE)
    print(f"Final test set performance for {n_components} PCA components: Loss {test_loss:.4f}, Accuracy {test_accuracy:.4f}")
    print(f"Early stopping at epoch: {early_stopping_epoch}")
    print()
    return test_accuracy, test_loss, early_stopping_epoch

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
DEVICE = torch.device("cpu")

import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.optim import Adam
from torch.nn.functional import relu, softmax
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
from sklearn.decomposition import PCA

def train(net, trainloader, valloader, testloader, epochs, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

    best_valid_loss = float('inf')
    best_model = None
    best_test_loss = float('inf')
    best_test_accuracy = 0.0
    patience_counter = 0
    early_stopping_epoch = 0

    # Set requires_grad = True for all model parameters
    for param in net.parameters():
        param.requires_grad = True

    for epoch in range(epochs):
        net.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)

            # Check if the loss tensor requires gradients
            if not loss.requires_grad:
                print("Warning: Loss tensor does not require gradients.")
                print("Skipping backward pass for this batch.")
                continue

            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(trainloader)
        train_acc = correct / total

        net.eval()
        running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in valloader:
                images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        valid_loss = running_loss / len(valloader)
        valid_acc = correct / total

        # Calculate test metrics for every epoch
        with torch.no_grad():
            test_loss, test_accuracy = test(net, testloader)

        # Print metrics for every epoch
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.4f}")

        scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_model = net.state_dict()
            best_test_loss = test_loss
            best_test_accuracy = test_accuracy
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= 10:
            early_stopping_epoch = epoch + 1
            break

    net.load_state_dict(best_model)
    return net, early_stopping_epoch, best_test_loss, best_test_accuracy


def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy


In [ ]:

pca_components_range = [8, 16, 32, 64, 128]  # Define the range of PCA components to try

best_accuracy = 0
best_pca_components = None
best_test_loss = float('inf')
best_compression = 0

# ... (previous code remains the same)

# Split the dataset into NUM_CLIENTS partitions
partition_size = len(trainset) // NUM_CLIENTS
lengths = [partition_size] * NUM_CLIENTS
datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))


client_data_variabilities = []
for i in range(NUM_CLIENTS):
    data_variability = characterize_client_data(datasets[i].dataset.data.numpy())
    client_data_variabilities.append(data_variability)

# Determine PCA components dynamically based on client data variability
pca, n_components, avg_variability = determine_pca_components(mnist_dataset.data.numpy(), client_data_variabilities)

# Load the datasets using the PCA object
trainloaders, valloaders, testloader, datasets, testset, _, _ = load_datasets(pca, n_components)



results = []


for n_components in pca_components_range:
    print(f"Training with {n_components} PCA components...")
    pca, n_components, avg_variability = determine_pca_components(mnist_dataset.data.numpy(), client_data_variabilities, n_components_range=(n_components, n_components))
    print(f"Average variability: {avg_variability:.4f}")
    print(f"Determined number of PCA components: {n_components}")

    trainloaders, valloaders, testloader, datasets, testset, _, _ = load_datasets(pca, n_components)

    train_removed_percentages = []
    test_removed_percentages = []
    for i in range(NUM_CLIENTS):
        _, train_removed_percentage = apply_pca_locally(datasets[i].dataset.data.numpy(), pca, n_components)
        _, test_removed_percentage = apply_pca_locally(testset.data.numpy(), pca, n_components)
        train_removed_percentages.append(train_removed_percentage)
        test_removed_percentages.append(test_removed_percentage)

    # Determine compression parameters based on client data variability
    pruning_amount, quantization_type = determine_compression_parameters(client_data_variabilities)

    trainloader = trainloaders[0]
    valloader = valloaders[0]

    test_accuracy, test_loss, early_stopping_epoch = train_and_evaluate(pca, n_components, trainloader, valloader, testloader)

    compression_ratio = (1 - (n_components * 784) / (28 * 28 * 784)) * 100

    results.append({
        'n_components': n_components,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'compression_ratio': compression_ratio
    })

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_pca_components = n_components
        best_test_loss = test_loss
        best_compression = compression_ratio
        best_early_stopping_epoch = early_stopping_epoch


print(f"Best PCA Components: {best_pca_components}")
print(f"Best Test Accuracy: {best_accuracy:.4f}")
print(f"Best Test Loss: {best_test_loss:.4f}")
print(f"Best Early Stopping Epoch: {best_early_stopping_epoch}")
print(f"Quantization Type: {quantization_type}")
print(f"Pruning Amount: {pruning_amount}")



print("Results:")
for result in results:
    print(f"PCA Components: {result['n_components']}, Test Loss: {result['test_loss']:.4f}, Test Accuracy: {result['test_accuracy']:.4f}, Compression Ratio: {result['compression_ratio']:.2f}%")

Training with 8 PCA components...
Average variability: 2867.7677
Determined number of PCA components: 8
Epoch 1/100 - Train Loss: 0.6457, Train Acc: 0.7975, Valid Loss: 0.1498, Valid Acc: 0.9681, Test Loss: 0.0106, Test Acc: 0.9082
Epoch 2/100 - Train Loss: 0.1626, Train Acc: 0.9486, Valid Loss: 0.0801, Valid Acc: 0.9815, Test Loss: 0.0060, Test Acc: 0.9408
Epoch 3/100 - Train Loss: 0.0969, Train Acc: 0.9713, Valid Loss: 0.0801, Valid Acc: 0.9782, Test Loss: 0.0057, Test Acc: 0.9499
Epoch 4/100 - Train Loss: 0.0781, Train Acc: 0.9765, Valid Loss: 0.0636, Valid Acc: 0.9799, Test Loss: 0.0051, Test Acc: 0.9554
Epoch 5/100 - Train Loss: 0.0590, Train Acc: 0.9799, Valid Loss: 0.0540, Valid Acc: 0.9866, Test Loss: 0.0041, Test Acc: 0.9617
Epoch 6/100 - Train Loss: 0.0463, Train Acc: 0.9844, Valid Loss: 0.0467, Valid Acc: 0.9849, Test Loss: 0.0043, Test Acc: 0.9622
Epoch 7/100 - Train Loss: 0.0330, Train Acc: 0.9886, Valid Loss: 0.0675, Valid Acc: 0.9799, Test Loss: 0.0050, Test Acc: 0.9567


In [ ]:
import io

# Model size calculation function
def get_model_size(model):
    buffer = io.BytesIO()
    torch.save(model.state_dict(), buffer)
    size = buffer.tell()  # Size in bytes
    return size

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import time
import numpy as np
from collections import OrderedDict
import torch

def get_parameters(net) -> List[np.ndarray]:
    parameters = []
    for name, param in net.named_parameters():
        print(f"Get parameters: {name}: {param.shape}")
        parameters.append(param.detach().cpu().numpy())
    return parameters

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})

    # Print the expected parameter shapes and the received parameter shapes
    print("Expected parameter shapes:")
    for name, param in net.named_parameters():
        print(f"{name}: {param.shape}")

    print("Received parameter shapes:")
    for name, param in state_dict.items():
        print(f"{name}: {param.shape}")

    # Try to load the state dictionary and handle shape mismatch
    try:
        net.load_state_dict(state_dict, strict=True)
    except RuntimeError as e:
        print("Shape mismatch occurred. Attempting to handle gracefully...")

        # Create a new state dictionary with matching shapes
        new_state_dict = OrderedDict()
        for (name, param), (state_name, state_param) in zip(net.named_parameters(), state_dict.items()):
            if param.shape != state_param.shape:
                print(f"Shape mismatch for parameter '{name}'. Expected {param.shape}, got {state_param.shape}.")
                if param.numel() == state_param.numel():
                    # Reshape the parameter if the total number of elements matches
                    state_param = state_param.view(param.shape)
                else:
                    print(f"Cannot reshape parameter '{name}' due to incompatible number of elements.")
                    continue
            new_state_dict[name] = state_param

        # Load the new state dictionary
        net.load_state_dict(new_state_dict, strict=False)
        print("Shape mismatch handled.")




In [ ]:

import torch.quantization
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader, Subset
import numpy as np

def apply_pruning(model, pruning_technique, amount):
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            if pruning_technique == 'l1_unstructured':
                prune.l1_unstructured(module, name='weight', amount=amount)
            elif pruning_technique == 'l1_structured':
                prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)
            elif pruning_technique == 'global_unstructured':
                prune.global_unstructured(module, name='weight', amount=amount)
            else:
                raise ValueError(f"Unsupported pruning technique: {pruning_technique}")
            prune.remove(module, 'weight')

def apply_quantization(model, quantization_type):
    if quantization_type == 'dynamic':
        quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
    elif quantization_type == 'static':
        model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
        torch.quantization.prepare(model, inplace=True)
        torch.quantization.convert(model, inplace=True)
        quantized_model = model
    else:
        raise ValueError(f"Unsupported quantization type: {quantization_type}")
    return quantized_model


In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch

def fine_tune(net, trainloader, valloader, device):
    # Define the loss function and the optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4)
    # Initialize the learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

    for epoch in range(3):  # Run for 3 epochs
        net.train()  # Set the network to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in trainloader:  # Iterate over the training data
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = net(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(trainloader)
        train_acc = correct / total

        net.eval()  # Set the network to evaluation mode
        running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # Disable gradient computation
            for images, labels in valloader:  # Iterate over the validation data
                images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        valid_loss = running_loss / len(valloader)
        valid_acc = correct / total

        # Print the training and validation statistics
        print(f"Epoch {epoch+1}/3 - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

        scheduler.step(valid_loss)  # Adjust the learning rate based on the validation loss

    return net  # Return the trained network

In [ ]:
DEVICE = torch.device("cpu")

def compress_model(net, pruning_technique, pruning_amount, quantization_type):
    # Apply pruning
    apply_pruning(net, pruning_technique, pruning_amount)

    # Re-enable gradients for the remaining parameters
    for param in net.parameters():
        param.requires_grad = True

    # Fine-tune the pruned model for 3 epochs
    fine_tune(net, trainloader, valloader, DEVICE)

    # Apply quantization
    quantized_model = apply_quantization(net, quantization_type)

    return quantized_model

In [ ]:
def determine_compression_parameters(client_data_variabilities):
    avg_variability = np.mean(client_data_variabilities)
    pruning_amount = np.interp(avg_variability, [0, 1], [0.2, 0.8])
    quantization_type = 'dynamic' if avg_variability > 0.5 else 'static'
    return pruning_amount, quantization_type

In [ ]:
import time
import numpy as np
from collections import OrderedDict
import torch
import flwr as fl

DEVICE = torch.device("cpu")

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader, testloader, pca, train_removed_percentage, test_removed_percentage, early_stopping_epoch, n_components, available_bandwidth=1000, pruning_amount=None, quantization_type=None):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.testloader = testloader
        self.pca = pca
        self.train_removed_percentage = train_removed_percentage
        self.test_removed_percentage = test_removed_percentage
        self.early_stopping_epoch = early_stopping_epoch
        self.n_components = n_components
        self.original_model_size = get_model_size(net)
        self.communication_rounds = 0
        self.communication_cost = 0
        self.bandwidth_utilization = 0
        self.latency = 0
        self.available_bandwidth = float(available_bandwidth)
        self.pruning_amount = pruning_amount
        self.quantization_type = quantization_type


    def get_parameters(self, **kwargs):
        parameters = get_parameters(self.net)
        return parameters

    def set_parameters(self, parameters):
        set_parameters(self.net, parameters)

    def calculate_bandwidth_utilization(self, model_size_before_compression, model_size_after_compression):
        # Calculate the size of the data sent and received in bytes
        data_sent = model_size_after_compression
        data_received = model_size_after_compression

        # Calculate the total data exchanged
        total_data = data_sent + data_received

        # Convert the total data to megabits (Mb)
        total_data_mb = total_data / (1024 * 1024) * 8

        # Calculate the bandwidth utilization percentage
        bandwidth_utilization = (total_data_mb / float(self.available_bandwidth)) * 100

        return bandwidth_utilization

    def fit(self, parameters, config):
        start_time = time.time()
        self.set_parameters(parameters)
        epochs = config.get("epochs", self.early_stopping_epoch)

        if self.pruning_amount is None or self.quantization_type is None:
            self.pruning_amount, self.quantization_type = determine_compression_parameters(client_data_variabilities)

        model_size_before_compression = get_model_size(self.net)
        compressed_model = compress_model(self.net, pruning_technique='l1_unstructured', pruning_amount=self.pruning_amount, quantization_type=self.quantization_type)
        model_size_after_compression = get_model_size(compressed_model)

        # Enable gradients for the compressed model parameters
        for param in compressed_model.parameters():
            param.requires_grad = True

        compressed_model, _, _, _ = train(compressed_model, self.trainloader, self.valloader, self.testloader, epochs, DEVICE)

        reduced_size = model_size_after_compression
        compression_reduction_percentage = (1 - reduced_size / model_size_before_compression) * 100

        self.communication_rounds += 1
        self.communication_cost += reduced_size * (1 - self.train_removed_percentage / 100) * (1 - compression_reduction_percentage / 100)
        self.bandwidth_utilization += self.calculate_bandwidth_utilization(model_size_before_compression, model_size_after_compression) * (1 - self.train_removed_percentage / 100) * (1 - compression_reduction_percentage / 100)
        self.latency += (time.time() - start_time) * (1 - self.train_removed_percentage / 100) * (1 - compression_reduction_percentage / 100)

        return get_parameters(compressed_model), len(self.trainloader.dataset), {
            "pca_reduction_percentage": self.train_removed_percentage,
            "compression_reduction_percentage": compression_reduction_percentage,
            "n_components": self.n_components,
            "communication_rounds": self.communication_rounds,
            "communication_cost": self.communication_cost,
            "bandwidth_utilization": self.bandwidth_utilization,
            "latency": self.latency,
        }

    def evaluate(self, parameters, config):
        start_time = time.time()
        self.set_parameters(parameters)

        if self.pruning_amount is None or self.quantization_type is None:
            # Determine compression parameters based on client data variability
            self.pruning_amount, self.quantization_type = determine_compression_parameters(client_data_variabilities)

        model_size_before_compression = get_model_size(self.net)
        compressed_model = compress_model(self.net, pruning_technique='l1_unstructured', pruning_amount=self.pruning_amount, quantization_type=self.quantization_type)
        model_size_after_compression = get_model_size(compressed_model)

        reduced_size = model_size_after_compression
        compression_reduction_percentage = (1 - reduced_size / model_size_before_compression) * 100
        loss, accuracy = test(compressed_model, self.testloader)

        self.communication_rounds += 1
        self.communication_cost += reduced_size * (1 - self.test_removed_percentage / 100) * (1 - compression_reduction_percentage / 100)
        self.bandwidth_utilization += self.calculate_bandwidth_utilization(model_size_before_compression, model_size_after_compression) * (1 - self.test_removed_percentage / 100) * (1 - compression_reduction_percentage / 100)
        self.latency += (time.time() - start_time) * (1 - self.test_removed_percentage / 100) * (1 - compression_reduction_percentage / 100)

        return float(loss), len(self.testloader.dataset), {
            "accuracy": float(accuracy),
            "pca_reduction_percentage": float(self.test_removed_percentage),
            "compression_reduction_percentage": compression_reduction_percentage,
            "n_components": float(self.n_components),
            "communication_rounds": float(self.communication_rounds),
            "communication_cost": float(self.communication_cost),
            "bandwidth_utilization": float(self.bandwidth_utilization),
            "latency": float(self.latency),
        }




In [ ]:

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Extract PCA reduction percentages from metrics
    pca_reduction_percentages = [m["pca_reduction_percentage"] for _, m in metrics]

    # Extract compression reduction percentages from metrics
    compression_reduction_percentages = [m["compression_reduction_percentage"] for _, m in metrics]

    communication_rounds = [m["communication_rounds"] for _, m in metrics]
    communication_cost = [m["communication_cost"] for _, m in metrics]
    bandwidth_utilization = [m["bandwidth_utilization"] for _, m in metrics]
    latency = [m["latency"] for _, m in metrics]

    return {
        "accuracy": sum(accuracies) / sum(examples),
        "pca_reduction_percentage": sum(pca_reduction_percentages) / len(pca_reduction_percentages),
        "compression_reduction_percentage": sum(compression_reduction_percentages) / len(compression_reduction_percentages),
        "communication_rounds": sum(communication_rounds) / len(communication_rounds),
        "communication_cost": sum(communication_cost),
        "bandwidth_utilization": sum(bandwidth_utilization) / len(bandwidth_utilization),
        "latency": sum(latency) / len(latency),
    }

In [ ]:
# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

mnist_dataset = MNIST("./dataset", train=True, download=True, transform=transform)


# Split the dataset into training (80%) and testing (20%)
train_size = int(0.8 * len(mnist_dataset))
test_size = len(mnist_dataset) - train_size
train_dataset, test_dataset = random_split(mnist_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))


def client_fn(cid, pca, train_removed_percentage, test_removed_percentage, testloader, n_components):
    model = Net().to(DEVICE)
    trainloaders, valloaders, testloader, datasets, testset, train_removed_percentage, test_removed_percentage = load_datasets(pca, n_components)
    pruning_amount, quantization_type = determine_compression_parameters(client_data_variabilities)
    client = FlowerClient(model, trainloaders[int(cid)], valloaders[int(cid)], testloader, pca, train_removed_percentage, test_removed_percentage, best_early_stopping_epoch, n_components, available_bandwidth=1000, pruning_amount=pruning_amount, quantization_type=quantization_type)
    return client.to_client()

# Characterize client data variability
client_data_variabilities = []
for i in range(NUM_CLIENTS):
    data_variability = characterize_client_data(datasets[i].dataset.data.numpy())
    client_data_variabilities.append(data_variability)

# Determine PCA components dynamically based on client data variability
pca, n_components, avg_variability = determine_pca_components(mnist_dataset.data.numpy(), client_data_variabilities)

# Load the datasets using the PCA object
trainloaders, valloaders, testloader, datasets, testset, _, _ = load_datasets(pca, n_components)

# Determine compression parameters based on client data variability
pruning_amount, quantization_type = determine_compression_parameters(client_data_variabilities)


pca_components_range = [8, 16, 32, 64, 128]
best_accuracy = 0
best_pca_components = None
best_test_loss = float('inf')
best_early_stopping_epoch = 0

for n_components in pca_components_range:
    print(f"Training with {n_components} PCA components...")
    pca, n_components, avg_variability = determine_pca_components(mnist_dataset.data.numpy(), client_data_variabilities, n_components_range=(n_components, n_components))
    print(f"Average variability: {avg_variability:.4f}")
    print(f"Determined number of PCA components: {n_components}")

    trainloaders, valloaders, testloader, datasets, testset, _, _ = load_datasets(pca, n_components)

    # Determine compression parameters based on client data variability
    pruning_amount, quantization_type = determine_compression_parameters(client_data_variabilities)


    train_removed_percentages = []
    test_removed_percentages = []
    for i in range(NUM_CLIENTS):
        _, train_removed_percentage = apply_pca_locally(datasets[i].dataset.data.numpy(), pca, n_components)
        _, test_removed_percentage = apply_pca_locally(testset.data.numpy(), pca, n_components)
        train_removed_percentages.append(train_removed_percentage)
        test_removed_percentages.append(test_removed_percentage)

    trainloader = trainloaders[0]
    valloader = valloaders[0]

    test_accuracy, test_loss, early_stopping_epoch = train_and_evaluate(pca, n_components, trainloader, valloader, testloader)

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_pca_components = n_components
        best_test_loss = test_loss
        best_early_stopping_epoch = early_stopping_epoch


print(f"Best PCA Components: {best_pca_components}")
print(f"Best Test Accuracy: {best_accuracy:.4f}")
print(f"Best Test Loss: {best_test_loss:.4f}")
print(f"Best Early Stopping Epoch: {best_early_stopping_epoch}")
print(f"Quantization Type: {quantization_type}")
print(f"Pruning Amount: {pruning_amount}")

# Pass the PCA object, data removed percentages, testloader, number of components, and compression parameters to the client_fn
def client_fn_with_pca(cid):
    return client_fn(cid, pca, train_removed_percentages[int(cid)], test_removed_percentages[int(cid)], testloader, best_pca_components)

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=10,
    evaluate_metrics_aggregation_fn=weighted_average,
)


# ... (previous code remains the same)

metrics_history = {}  # Dictionary to store metrics for each epoch

try:
    history = fl.simulation.start_simulation(
        client_fn=client_fn_with_pca,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=best_early_stopping_epoch),
        strategy=strategy,
    )

    if history is not None and history.metrics_distributed:
        for epoch, epoch_metrics in enumerate(history.metrics_distributed, start=1):
            if epoch_metrics:
                epoch_accuracies = [client[1].get("accuracy", 0.0) for client in epoch_metrics]
                epoch_losses = [client[1].get("loss", 0.0) for client in epoch_metrics]
                epoch_communication_costs = [client[1].get("communication_cost", 0.0) for client in epoch_metrics]
                epoch_latencies = [client[1].get("latency", 0.0) for client in epoch_metrics]
                epoch_bandwidth_utilizations = [client[1].get("bandwidth_utilization", 0.0) for client in epoch_metrics]
                epoch_compression_reduction_percentages = [client[1].get("compression_reduction_percentage", 0.0) for client in epoch_metrics]
                epoch_train_accuracies = [client[1].get("train_accuracy", 0.0) for client in epoch_metrics]
                epoch_train_losses = [client[1].get("train_loss", 0.0) for client in epoch_metrics]
                epoch_test_accuracies = [client[1].get("test_accuracy", 0.0) for client in epoch_metrics]
                epoch_test_losses = [client[1].get("test_loss", 0.0) for client in epoch_metrics]

                metrics_history[epoch] = {
                    "accuracy": sum(epoch_accuracies) / len(epoch_accuracies),
                    "loss": sum(epoch_losses) / len(epoch_losses),
                    "communication_cost": sum(epoch_communication_costs),
                    "latency": sum(epoch_latencies) / len(epoch_latencies),
                    "bandwidth_utilization": sum(epoch_bandwidth_utilizations) / len(epoch_bandwidth_utilizations),
                    "compression_reduction_percentage": sum(epoch_compression_reduction_percentages) / len(epoch_compression_reduction_percentages),
                    "train_accuracy": sum(epoch_train_accuracies) / len(epoch_train_accuracies),
                    "train_loss": sum(epoch_train_losses) / len(epoch_train_losses),
                    "test_accuracy": sum(epoch_test_accuracies) / len(epoch_test_accuracies),
                    "test_loss": sum(epoch_test_losses) / len(epoch_test_losses),
                }
    else:
        print("No metrics available.")
except Exception as e:
    print(f"Simulation failed with error: {str(e)}")
    # Handle the exception appropriately

# ... (previous code remains the same)

print(f"Average metrics for {best_pca_components} PCA components:")

for epoch in range(1, best_early_stopping_epoch + 1):
    if epoch in metrics_history:
        epoch_metrics = metrics_history[epoch]

        avg_accuracy = epoch_metrics["accuracy"]
        avg_loss = epoch_metrics["loss"]
        avg_communication_cost = epoch_metrics["communication_cost"]
        avg_latency = epoch_metrics["latency"]
        avg_bandwidth_utilization = epoch_metrics["bandwidth_utilization"]
        avg_compression_reduction_percentage = epoch_metrics["compression_reduction_percentage"]
        avg_train_accuracy = epoch_metrics["train_accuracy"]
        avg_train_loss = epoch_metrics["train_loss"]
        avg_test_accuracy = epoch_metrics["test_accuracy"]
        avg_test_loss = epoch_metrics["test_loss"]

        print(f"Epoch {epoch}:")
        print(f"  Average Accuracy: {avg_accuracy:.4f}")
        print(f"  Average Loss: {avg_loss:.4f}")
        print(f"  Average Communication Cost: {avg_communication_cost:.2f} bytes")
        print(f"  Average Latency: {avg_latency:.2f} seconds")
        print(f"  Average Bandwidth Utilization: {avg_bandwidth_utilization:.2f}%")
        print(f"  Average Compression Reduction Percentage: {avg_compression_reduction_percentage:.2f}%")
        print(f"  Average Train Accuracy: {avg_train_accuracy:.4f}")
        print(f"  Average Train Loss: {avg_train_loss:.4f}")
        print(f"  Average Test Accuracy: {avg_test_accuracy:.4f}")
        print(f"  Average Test Loss: {avg_test_loss:.4f}")
        print()
    else:
        print(f"No metrics available for Epoch {epoch}")
        print()

# Print the metrics for each epoch
for epoch in range(1, best_early_stopping_epoch + 1):
    if epoch in metrics_history:
        metrics = metrics_history[epoch]
        print(f"Epoch {epoch}:")
        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        print(f"  Loss: {metrics['loss']:.4f}")
        print(f"  Communication Cost: {metrics['communication_cost']:.2f} bytes")
        print(f"  Latency: {metrics['latency']:.2f} seconds")
        print(f"  Bandwidth Utilization: {metrics['bandwidth_utilization']:.2f}%")
        print(f"  Compression Reduction Percentage: {metrics['compression_reduction_percentage']:.2f}%")
        print(f"  Train Accuracy: {metrics['train_accuracy']:.4f}")
        print(f"  Train Loss: {metrics['train_loss']:.4f}")
        print(f"  Test Accuracy: {metrics['test_accuracy']:.4f}")
        print(f"  Test Loss: {metrics['test_loss']:.4f}")
        print()
    else:
        print(f"No metrics available for Epoch {epoch}")
        print()


Training with 8 PCA components...
Average variability: 2867.7677
Determined number of PCA components: 8
Epoch 1/100 - Train Loss: 0.7079, Train Acc: 0.7874, Valid Loss: 0.2165, Valid Acc: 0.9396, Test Loss: 0.0151, Test Acc: 0.8567
Epoch 2/100 - Train Loss: 0.1816, Train Acc: 0.9437, Valid Loss: 0.1002, Valid Acc: 0.9715, Test Loss: 0.0084, Test Acc: 0.9188
Epoch 3/100 - Train Loss: 0.1093, Train Acc: 0.9670, Valid Loss: 0.0673, Valid Acc: 0.9815, Test Loss: 0.0053, Test Acc: 0.9489
Epoch 4/100 - Train Loss: 0.0794, Train Acc: 0.9760, Valid Loss: 0.0579, Valid Acc: 0.9799, Test Loss: 0.0054, Test Acc: 0.9502
Epoch 5/100 - Train Loss: 0.0579, Train Acc: 0.9812, Valid Loss: 0.0537, Valid Acc: 0.9883, Test Loss: 0.0037, Test Acc: 0.9632
Epoch 6/100 - Train Loss: 0.0493, Train Acc: 0.9842, Valid Loss: 0.0400, Valid Acc: 0.9916, Test Loss: 0.0043, Test Acc: 0.9603
Epoch 7/100 - Train Loss: 0.0354, Train Acc: 0.9896, Valid Loss: 0.0769, Valid Acc: 0.9698, Test Loss: 0.0049, Test Acc: 0.9547


INFO :      Starting Flower simulation, config: num_rounds=37, no round_timeout
INFO:flwr:Starting Flower simulation, config: num_rounds=37, no round_timeout


Epoch 16/100 - Train Loss: 0.0057, Train Acc: 0.9983, Valid Loss: 0.0724, Valid Acc: 0.9815, Test Loss: 0.0032, Test Acc: 0.9732
Final test set performance for 128 PCA components: Loss 0.0037, Accuracy 0.9666
Early stopping at epoch: 16

Best PCA Components: 8
Best Test Accuracy: 0.9749
Best Test Loss: 0.0034
Best Early Stopping Epoch: 37
Quantization Type: dynamic
Pruning Amount: 0.8


2024-04-17 01:42:49,874	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3918724300.0, 'memory': 7837448603.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3918724300.0, 'memory': 7837448603.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO:flwr:No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO:flwr:Flower VCE: Resources for each Virtual Client: {'num_cpus': 1

(ClientAppActor pid=16053) Get parameters: conv1.weight: torch.Size([16, 1, 3, 3])
(ClientAppActor pid=16053) Get parameters: conv1.bias: torch.Size([16])
(ClientAppActor pid=16053) Get parameters: conv2.weight: torch.Size([32, 16, 3, 3])
(ClientAppActor pid=16053) Get parameters: conv2.bias: torch.Size([32])
(ClientAppActor pid=16053) Get parameters: fc1.weight: torch.Size([128, 1568])
(ClientAppActor pid=16053) Get parameters: fc1.bias: torch.Size([128])
(ClientAppActor pid=16053) Get parameters: fc2.weight: torch.Size([10, 128])
(ClientAppActor pid=16053) Get parameters: fc2.bias: torch.Size([10])
(ClientAppActor pid=16053) Expected parameter shapes:
(ClientAppActor pid=16053) conv1.weight: torch.Size([16, 1, 3, 3])
(ClientAppActor pid=16053) conv1.bias: torch.Size([16])
(ClientAppActor pid=16053) conv2.weight: torch.Size([32, 16, 3, 3])
(ClientAppActor pid=16053) conv2.bias: torch.Size([32])
(ClientAppActor pid=16053) fc1.weight: torch.Size([128, 1568])
(ClientAppActor pid=16053) f

...

(ClientAppActor pid=16051) Epoch 1/37 - Train Loss: 0.5155, Train Acc: 0.8271, Valid Loss: 0.4428, Valid Acc: 0.8615, Test Loss: 0.0090, Test Acc: 0.9125
(ClientAppActor pid=16053) Epoch 3/3 - Train Loss: 0.2104, Train Acc: 0.9367, Valid Loss: 0.1471, Valid Acc: 0.9648
(ClientAppActor pid=16051) Epoch 2/37 - Train Loss: 0.5048, Train Acc: 0.8320, Valid Loss: 0.4428, Valid Acc: 0.8615, Test Loss: 0.0090, Test Acc: 0.9125 [repeated 2x across cluster]
(ClientAppActor pid=16051) Epoch 3/37 - Train Loss: 0.5187, Train Acc: 0.8391, Valid Loss: 0.4428, Valid Acc: 0.8615, Test Loss: 0.0090, Test Acc: 0.9125 [repeated 2x across cluster]
(ClientAppActor pid=16051) Epoch 4/37 - Train Loss: 0.5098, Train Acc: 0.8359, Valid Loss: 0.4428, Valid Acc: 0.8615, Test Loss: 0.0090, Test Acc: 0.9125 [repeated 2x across cluster]
(ClientAppActor pid=16051) Epoch 5/37 - Train Loss: 0.5173, Train Acc: 0.8308, Valid Loss: 0.4428, Valid Acc: 0.8615, Test Loss: 0.0090, Test Acc: 0.9125 [repeated 2x across cluster

(ClientAppActor pid=16051) /usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:266: UserWarning: quantized::linear_dynamic: an autograd kernel was not registered to the Autograd key(s) but we are trying to backprop through it. This may lead to silently incorrect behavior. This behavior is deprecated and will be removed in a future version of PyTorch. If your operator is differentiable, please ensure you have registered an autograd kernel to the correct Autograd key (e.g. DispatchKey::Autograd, DispatchKey::CompositeImplicitAutograd). If your operator is not differentiable, or to squash this warning and use the previous behavior, please register torch::CppFunction::makeFallthrough() to DispatchKey::Autograd. (Triggered internally at ../torch/csrc/autograd/autograd_not_implemented_fallback.cpp:63.) [repeated 2x across cluster]
(ClientAppActor pid=16051)   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass [repeated 2x across clus

(ClientAppActor pid=16051) Epoch 1/37 - Train Loss: 0.2998, Train Acc: 0.9141, Valid Loss: 0.2720, Valid Acc: 0.9263, Test Loss: 0.0065, Test Acc: 0.9411
(ClientAppActor pid=16053) Epoch 3/3 - Train Loss: 0.1947, Train Acc: 0.9402, Valid Loss: 0.1303, Valid Acc: 0.9698
(ClientAppActor pid=16051) Epoch 2/37 - Train Loss: 0.2946, Train Acc: 0.9158, Valid Loss: 0.2720, Valid Acc: 0.9263, Test Loss: 0.0065, Test Acc: 0.9411 [repeated 2x across cluster]
(ClientAppActor pid=16051) Epoch 3/37 - Train Loss: 0.2897, Train Acc: 0.9149, Valid Loss: 0.2720, Valid Acc: 0.9263, Test Loss: 0.0065, Test Acc: 0.9411 [repeated 2x across cluster]
(ClientAppActor pid=16051) Epoch 4/37 - Train Loss: 0.2950, Train Acc: 0.9164, Valid Loss: 0.2720, Valid Acc: 0.9263, Test Loss: 0.0065, Test Acc: 0.9411 [repeated 2x across cluster]
(ClientAppActor pid=16051) Epoch 5/37 - Train Loss: 0.2896, Train Acc: 0.9156, Valid Loss: 0.2720, Valid Acc: 0.9263, Test Loss: 0.0065, Test Acc: 0.9411 [repeated 2x across cluster

(ClientAppActor pid=16051) /usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:266: UserWarning: quantized::linear_dynamic: an autograd kernel was not registered to the Autograd key(s) but we are trying to backprop through it. This may lead to silently incorrect behavior. This behavior is deprecated and will be removed in a future version of PyTorch. If your operator is differentiable, please ensure you have registered an autograd kernel to the correct Autograd key (e.g. DispatchKey::Autograd, DispatchKey::CompositeImplicitAutograd). If your operator is not differentiable, or to squash this warning and use the previous behavior, please register torch::CppFunction::makeFallthrough() to DispatchKey::Autograd. (Triggered internally at ../torch/csrc/autograd/autograd_not_implemented_fallback.cpp:63.) [repeated 2x across cluster]
(ClientAppActor pid=16051)   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass [repeated 2x across clus

(ClientAppActor pid=16051) Epoch 1/37 - Train Loss: 0.2792, Train Acc: 0.9197, Valid Loss: 0.2004, Valid Acc: 0.9396, Test Loss: 0.0081, Test Acc: 0.9231
(ClientAppActor pid=16051) Epoch 3/3 - Train Loss: 0.1702, Train Acc: 0.9490, Valid Loss: 0.1200, Valid Acc: 0.9614
(ClientAppActor pid=16051) Epoch 2/37 - Train Loss: 0.2751, Train Acc: 0.9189, Valid Loss: 0.2004, Valid Acc: 0.9396, Test Loss: 0.0081, Test Acc: 0.9231 [repeated 2x across cluster]
(ClientAppActor pid=16051) Epoch 3/37 - Train Loss: 0.2711, Train Acc: 0.9204, Valid Loss: 0.2004, Valid Acc: 0.9396, Test Loss: 0.0081, Test Acc: 0.9231 [repeated 2x across cluster]
(ClientAppActor pid=16051) Epoch 4/37 - Train Loss: 0.2678, Train Acc: 0.9200, Valid Loss: 0.2004, Valid Acc: 0.9396, Test Loss: 0.0081, Test Acc: 0.9231 [repeated 2x across cluster]
(ClientAppActor pid=16051) Epoch 5/37 - Train Loss: 0.2741, Train Acc: 0.9165, Valid Loss: 0.2004, Valid Acc: 0.9396, Test Loss: 0.0081, Test Acc: 0.9231 [repeated 2x across cluster